In [1]:
##############################
# 2. SETUP & CONFIGURATION
##############################

import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 2.A: SET THE MLFLOW TRACKING URI
# Point MLflow to the local server started by Docker
mlflow.set_tracking_uri("http://192.168.1.44:5000")

# 2.B: OPTIONAL - Set a specific experiment name
EXPERIMENT_NAME = "my-classification-experiment"

experiment_id = None

# Try to get or create the experiment
try:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
except mlflow.exceptions.RestException:
    # If it already exists, just fetch its ID
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

print("MLflow Tracking URI:", mlflow.get_tracking_uri())
print("Experiment ID:", experiment_id)


MLflow Tracking URI: http://192.168.1.44:5000
Experiment ID: 996300549925381195


In [6]:
##############################
# 3. TRAIN A MODEL & LOG TO MLFLOW
##############################

# Load sample IRIS data
iris = load_iris()
X = iris.data
y = iris.target

# Split data for training & validation
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.2, 
                                                  random_state=42)

# Start an MLflow run
with mlflow.start_run(experiment_id=experiment_id, run_name="iris_rf_model") as run:

    # 3.A: Train a simple RandomForest
    n_estimators = 100
    max_depth = 5
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    clf.fit(X_train, y_train)

    # 3.B: Evaluate on validation set
    y_pred = clf.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    # 3.C: Log parameters & metrics
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_metric("accuracy", accuracy)

    # 3.D: Log the model itself (using mlflow.sklearn)
    mlflow.sklearn.log_model(sk_model=clf, artifact_path="model")

    # Printing info
    run_id = run.info.run_id
    print(f"Run ID: {run_id}")
    print(f"Logged Accuracy: {accuracy:.4f}")
    print("Model logged in the artifact store.")


2025/03/12 08:37:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: e77cec1a93fe413dbef5bd23c6a0545e
Logged Accuracy: 1.0000
Model logged in the artifact store.
🏃 View run iris_rf_model at: http://192.168.1.44:5000/#/experiments/996300549925381195/runs/e77cec1a93fe413dbef5bd23c6a0545e
🧪 View experiment at: http://192.168.1.44:5000/#/experiments/996300549925381195


In [5]:
##############################
# 4. REGISTER THE MODEL
##############################

# We'll give this model a name in the Model Registry
model_name = "iris_rf_registered_model"

# Path to the model artifact in the run above
# It's typically "runs:/{run_id}/model"
model_uri = f"runs:/{run_id}/model"

# We can register the model with mlflow.register_model
registered_model = mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)

print(f"Name: {registered_model.name}")
print(f"Version: {registered_model.version}")
print("Model registered in MLflow Model Registry!")


Registered model 'iris_rf_registered_model' already exists. Creating a new version of this model...
2025/03/12 08:36:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_rf_registered_model, version 2


Name: iris_rf_registered_model
Version: 2
Model registered in MLflow Model Registry!


Created version '2' of model 'iris_rf_registered_model'.


In [4]:
##############################
# 5. LOAD THE REGISTERED MODEL LOCALLY & PREDICT
##############################

# Once a model is registered, you can load it by name and version.
# For example, "models:/{model_name}/{version}"
# or "models:/{model_name}/Production" if you assign a stage.

from mlflow.pyfunc import load_model

model_version = registered_model.version  # or "1", "2", etc.
logged_model_uri = f"models:/{model_name}/{model_version}"

# Load the model
loaded_model = load_model(logged_model_uri)

# Perform predictions
sample_input = X_val[:5]  # first 5 examples from our validation set
predictions = loaded_model.predict(sample_input)

print("Sample input:\n", sample_input)
print("Predictions:\n", predictions)


Sample input:
 [[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]]
Predictions:
 [1 0 2 1 1]
